**NAME**           : De Guzman, Aero Kent D
\
**COURSE/SECTION** : CPE22S3 CPE311
\
**DATE CREATED**   : 04/11/2025
\
**DATE SUBMITTED** : 04/11/2025
\
**INSTRUCTOR**     : Engr. Roman Richard

#### PART 1

Exporting a Creating the Dataframe

In [75]:
# importing the needed libraries
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
print('import complete.')

import complete.


In [33]:
# choosing a website to scrape
response = get('https://www.ncaa.com/rankings/basketball-men/d1/ncaa-mens-basketball-net-rankings')
# print(response.text) # too many words,.. remove it if needed.

In [27]:
soup = BeautifulSoup(response.text, 'html.parser') # converting the converted html into a readable format in order to extract the data

amnt = soup.find_all('div', class_ = 'layout--has-sidebar') # this is to check how many tables are there inside,
print(type(amnt))                                     # this is 'optional' and can be go straight to data
print(len(amnt))                                      # creating the dataframe but it is great to know
                                                            # and prepare and select the needed frames

<class 'bs4.element.ResultSet'>
1


In [65]:
# ball # too many words again, remove it to see if needed.

In [470]:
# Extracting the headers.
# since it is only a single table, no need to specific which table is needed.

# extracting the columns
raw_head = soup.find_all('th')
head     = [h.text for h in raw_head]

# extracting the rows
df = pd.DataFrame(columns = head)
for row_data in soup.find_all('tr')[1:]:
    raw             = row_data.find_all('td')
    dat             = [val.text for val in raw]
    df.loc[len(df)] = dat

df.head(5)

,Rank,Previous,School,Conference,Record,Road,Neutral,WAB,Quad 1,Quad 2,Quad 3,Quad 4
0,1,1,Duke,ACC,31-3,10-1,4-2,6,9-3,7-0,10-0,5-0
1,2,2,Auburn,SEC,28-5,8-2,7-1,1,16-5,6-0,2-0,4-0
2,3,3,Houston,Big 12,30-4,10-0,4-3,3,14-3,6-1,5-0,5-0
3,4,4,Florida,SEC,30-4,7-3,8-0,2,11-4,9-0,4-0,6-0
4,5,5,Tennessee,SEC,27-7,6-5,5-1,5,11-7,5-0,5-0,6-0


#### Part 2

Preparing the Dataframe; cleaning and transforming any mistakes that could have appeared upon creation...

In [472]:
df.info() # to see both if there are any missing values as well as misaligned data types

<class 'pandas.core.frame.DataFrame'>
Index: 364 entries, 0 to 363
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rank        364 non-null    object
 1   Previous    364 non-null    object
 2   School      364 non-null    object
 3   Conference  364 non-null    object
 4   Record      364 non-null    object
 5   Road        364 non-null    object
 6   Neutral     364 non-null    object
 7   WAB         364 non-null    object
 8   Quad 1      364 non-null    object
 9   Quad 2      364 non-null    object
 10  Quad 3      364 non-null    object
 11  Quad 4      364 non-null    object
dtypes: object(12)
memory usage: 37.0+ KB


In [505]:
# correcting any mistaken data types
modified_df = df.copy()
modified_df[['Rank','Previous','WAB']] = modified_df[['Rank','Previous','WAB']].astype('int32')
print(modified_df.dtypes)
modified_df.columns

Rank           int32
Previous       int32
School        object
Conference    object
Record        object
Road          object
Neutral       object
WAB            int32
Quad 1        object
Quad 2        object
Quad 3        object
Quad 4        object
dtype: object


Index(['Rank', 'Previous', 'School', 'Conference', 'Record', 'Road', 'Neutral',
       'WAB', 'Quad 1', 'Quad 2', 'Quad 3', 'Quad 4'],
      dtype='object')

In [507]:
# Dividing/Separating the Record, Road, Neutral, WAB, Quad 1, Quad 2, Quad 3, Quad 4 into two to prepare for 
# future analysis...
df_winloss  = modified_df[['Record','Road','Neutral','Quad 1','Quad 2','Quad 3','Quad 4']] # choosing the ones to separate
for i in df_winloss.columns:
    modified_df[[f'{i} (wins)',f'{i} (loss)']] = modified_df[i].str.split('-', expand = True).astype('int32')
    modified_df.drop([i], inplace = True, axis = 1)

In [509]:
# original data frame
df

,Rank,Previous,School,Conference,Record,Road,Neutral,WAB,Quad 1,Quad 2,Quad 3,Quad 4
0,1,1,Duke,ACC,31-3,10-1,4-2,6,9-3,7-0,10-0,5-0
1,2,2,Auburn,SEC,28-5,8-2,7-1,1,16-5,6-0,2-0,4-0
2,3,3,Houston,Big 12,30-4,10-0,4-3,3,14-3,6-1,5-0,5-0
3,4,4,Florida,SEC,30-4,7-3,8-0,2,11-4,9-0,4-0,6-0
4,5,5,Tennessee,SEC,27-7,6-5,5-1,5,11-7,5-0,5-0,6-0
...,...,...,...,...,...,...,...,...,...,...,...,...
359,360,360,Alabama A&M,SWAC,7-22,2-12,0-1,336,0-1,0-3,0-3,7-15
360,361,361,UMES,MEAC,2-25,1-17,0-1,337,0-5,0-0,0-7,2-13
361,362,362,The Citadel,SoCon,1-25,0-12,0-1,356,0-1,0-4,0-10,1-10
362,363,363,Ark.-Pine Bluff,SWAC,4-25,2-17,0-0,346,0-5,0-0,0-1,4-19


In [511]:
# modified data frame
modified_df

,Rank,Previous,School,Conference,WAB,Record (wins),Record (loss),Road (wins),Road (loss),Neutral (wins),Neutral (loss),Quad 1 (wins),Quad 1 (loss),Quad 2 (wins),Quad 2 (loss),Quad 3 (wins),Quad 3 (loss),Quad 4 (wins),Quad 4 (loss)
0,1,1,Duke,ACC,6,31,3,10,1,4,2,9,3,7,0,10,0,5,0
1,2,2,Auburn,SEC,1,28,5,8,2,7,1,16,5,6,0,2,0,4,0
2,3,3,Houston,Big 12,3,30,4,10,0,4,3,14,3,6,1,5,0,5,0
3,4,4,Florida,SEC,2,30,4,7,3,8,0,11,4,9,0,4,0,6,0
4,5,5,Tennessee,SEC,5,27,7,6,5,5,1,11,7,5,0,5,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,360,360,Alabama A&M,SWAC,336,7,22,2,12,0,1,0,1,0,3,0,3,7,15
360,361,361,UMES,MEAC,337,2,25,1,17,0,1,0,5,0,0,0,7,2,13
361,362,362,The Citadel,SoCon,356,1,25,0,12,0,1,0,1,0,4,0,10,1,10
362,363,363,Ark.-Pine Bluff,SWAC,346,4,25,2,17,0,0,0,5,0,0,0,1,4,19


In [513]:
modified_df.to_csv('HOA 7p2 - deguzman - cpe22s3 cpe311.csv', index = False)

In [517]:
# checking if the .csv is correctly exported.
temp = pd.read_csv('HOA 7p2 - deguzman - cpe22s3 cpe311.csv')
temp

,Rank,Previous,School,Conference,WAB,Record (wins),Record (loss),Road (wins),Road (loss),Neutral (wins),Neutral (loss),Quad 1 (wins),Quad 1 (loss),Quad 2 (wins),Quad 2 (loss),Quad 3 (wins),Quad 3 (loss),Quad 4 (wins),Quad 4 (loss)
0,1,1,Duke,ACC,6,31,3,10,1,4,2,9,3,7,0,10,0,5,0
1,2,2,Auburn,SEC,1,28,5,8,2,7,1,16,5,6,0,2,0,4,0
2,3,3,Houston,Big 12,3,30,4,10,0,4,3,14,3,6,1,5,0,5,0
3,4,4,Florida,SEC,2,30,4,7,3,8,0,11,4,9,0,4,0,6,0
4,5,5,Tennessee,SEC,5,27,7,6,5,5,1,11,7,5,0,5,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,360,360,Alabama A&M,SWAC,336,7,22,2,12,0,1,0,1,0,3,0,3,7,15
360,361,361,UMES,MEAC,337,2,25,1,17,0,1,0,5,0,0,0,7,2,13
361,362,362,The Citadel,SoCon,356,1,25,0,12,0,1,0,1,0,4,0,10,1,10
362,363,363,Ark.-Pine Bluff,SWAC,346,4,25,2,17,0,0,0,5,0,0,0,1,4,19
